# HW 1 - Carlos Alvarado

### Problem 1 - CODE

In [1]:
import geopandas as gpd
import os
import pandas as pd
import requests
import datetime

from geopandas import GeoDataFrame
from geopandas.tools import sjoin
from shapely.geometry import Point

def get_and_load_data(data_info):
    '''
    Loads "data_name.csv" from local disk, or downloads it if it's not present
    
    Input: dictionary with information about required data
    Returns: pandas data_frame for "data_name"
    '''
    
    filepath = './data/{}.csv'.format(data_info['source'])
    
    if os.path.exists(filepath):
        data = pd.read_csv(filepath)
    else:
        data = helper_download_data(filepath, data_info['source'], data_info['datevar'])
    
    for old_var, new_var in data_info['rename']:
        data.rename(index=str, columns={old_var: new_var}, inplace=True)
        
    return data
    
def helper_download_data(filepath, data_name, datevar):
    offset = 0
    limit = 50000
    data = pd.DataFrame({})
    last_year = (datetime.datetime.now() - datetime.timedelta(days=365)).strftime('%Y-%m-%d')

    while True:
        
        url = 'https://data.cityofchicago.org/resource/' + \
              "{}.json?$order=service_request_number DESC &$limit={}&$offset={}&$where= {} > '{}'".format(
              data_name, limit, offset, datevar, last_year)
        print('getting data from', url)
    
        response = requests.get(url)
        response_code = response.status_code
        
        if response_code != 200: 
            print('Failed to download data')
            return data
            
        json_data = response.content

        if len(json_data) > 4:
            data = pd.concat([data, pd.read_json(json_data)])
        
            offset = offset + limit
        else:
            print(json_data)
            break
            
    #save data to csv for future use
    data.to_csv(filepath)
    
    return data    

In [9]:
# Loading complaints data. Added community names.

DATA1 = {'name': 'Graffiti Removal',
         'source': 'hec5-y4x5',
         'datevar': 'creation_date',
         'rename': [('where_is_the_graffiti_located_', 'Sub Type')]    
        }

DATA2 = {'name': 'Vacant and Abandoned Buildings Reported',
         'source': '7nii-7srd',
         'datevar': 'date_service_request_was_received',
         'rename': [('is_building_open_or_boarded_', 'Sub Type'), 
                    ('date_service_request_was_received', 'creation_date')]
        }

DATA3 = {'name': 'Pot Holes Reported',
         'source': '7as2-ds3y',
         'datevar': 'creation_date',
         'rename': [('zip', 'zip_code'), 
                    ('type_of_service_request', 'Sub Type')]
        }
    
DATA4 = {'name': 'Sanitation Code Complaints',
         'source': 'me59-5fac',
         'datevar': 'creation_date',         
         'rename': [('what_is_the_nature_of_this_code_violation_', 'Sub Type')]
        }

SOURCES = [DATA1, DATA2, DATA3, DATA4]

complaints = pd.DataFrame({})

for db_data in SOURCES:
    data = get_and_load_data(db_data)
    if not data.empty:
        print(data.columns)
        data['Complaint Type'] = db_data['name']
        complaints = pd.concat([complaints, data], ignore_index=True)

area_names_file = "./data/boundaries/Boundaries - Community Areas (current).geojson"
area_names = gpd.read_file(area_names_file)
area_names['area_numbe'] = pd.to_numeric(area_names.area_numbe, errors='coerce')

complaints = complaints.merge(area_names, left_on='community_area', right_on='area_numbe', how='left')

#complaints = pd.read_csv('./data/sample.csv')

clean_data = complaints[complaints['latitude'].notnull() & complaints['longitude'].notnull()]

geometry = [Point(xy) for xy in zip(clean_data.longitude, clean_data.latitude)]
clean_data = clean_data.drop(['latitude', 'longitude'], axis=1)

crs = {'init': 'epsg:4269'}
geo_complaints = GeoDataFrame(clean_data, crs=crs, geometry=geometry)


Index(['Unnamed: 0', 'community_area', 'completion_date', 'creation_date',
       'latitude', 'location', 'longitude', 'police_district',
       'service_request_number', 'ssa', 'status', 'street_address',
       'type_of_service_request', 'ward',
       'what_type_of_surface_is_the_graffiti_on_', 'Sub Type', 'x_coordinate',
       'y_coordinate', 'zip_code'],
      dtype='object')
Index(['Unnamed: 0', 'address_street_direction', 'address_street_name',
       'address_street_number', 'address_street_suffix',
       'any_people_using_property_homeless_childen_gangs_', 'community_area',
       'creation_date', 'if_the_building_is_open_where_is_the_entry_point_',
       'Sub Type', 'is_the_building_currently_vacant_or_occupied_',
       'is_the_building_vacant_due_to_fire_', 'latitude', 'location',
       'location_of_building_on_the_lot_if_garage_change_type_code_to_bgd_',
       'longitude', 'police_district', 'service_request_number',
       'service_request_type', 'ward', 'x_coordinat

In [ ]:
#create sample database to play with
complaints.sample(1000).to_csv('./data/sample.csv')

In [3]:
#number of complaints by type
ctcounts = complaints['Complaint Type'].value_counts().to_frame()
total = sum(ctcounts['Complaint Type'])
ctcounts['Percent'] = ctcounts['Complaint Type']/total
ctcounts

,Complaint Type,Percent
Graffiti Removal,116981,0.587531
Pot Holes Reported,55885,0.280680
Sanitation Code Complaints,21514,0.108053
Vacant and Abandoned Buildings Reported,4726,0.023736


In [4]:
# GRAPH NUMBER OF REQUESTS BY MONTH AND TYPE
from pandas import TimeGrouper
from matplotlib import pyplot
import numpy as np

n_per_month = complaints[['creation_date', 'Complaint Type', 'service_request_number']].groupby(['creation_date', 'Complaint Type'], as_index=False).count()
n_per_month = n_per_month.pivot(index='creation_date', columns='Complaint Type', values='service_request_number')
n_per_month = n_per_month.set_index(pd.DatetimeIndex(n_per_month.index))

n_per_month = n_per_month[n_per_month.index < '2017-04-01']

groups = n_per_month.groupby(TimeGrouper('M')).aggregate(np.sum)

groups.plot(figsize=(15,10), ylim=(0,15000), title='Number of requests by month and type')
pyplot.show()

In [29]:
gb = complaints[complaints['Complaint Type'] == 'Graffiti Removal']['Sub Type'].value_counts().to_frame()
total = sum(gb['Sub Type'])
gb['Percent'] = gb['Sub Type']/total
gb

,Sub Type,Percent
Front,35344,0.302911
Side,16797,0.143957
Pole,11129,0.095380
Garage,9897,0.084821
Rear,7644,0.065512
Alley,5928,0.050805
Traffic Control Box,5290,0.045337
Sign,4637,0.039741
Door,4008,0.034350
Dumpster,3770,0.032310


In [5]:
ctcounts = complaints['community'].value_counts().to_frame()
total = sum(ctcounts['community'])
ctcounts['Percent'] = ctcounts['community']/total
ctcounts[:10]

,community,Percent
WEST TOWN,12398,0.062354
LOGAN SQUARE,9629,0.048428
SOUTH LAWNDALE,6735,0.033873
BRIGHTON PARK,6615,0.033269
AVONDALE,6301,0.031690
LAKE VIEW,6226,0.031313
LOWER WEST SIDE,5993,0.030141
IRVING PARK,5845,0.029397
BELMONT CRAGIN,5098,0.025640
GAGE PARK,4881,0.024548


In [18]:
ctcounts['index'] = ctcounts.index
area_names2 = area_names.merge(ctcounts, left_on='community', right_on='index')
area_names2.plot(column='community_y', cmap='Blues')
pyplot.show()

In [105]:
complaints['Response Time'] = pd.to_numeric(pd.to_datetime(complaints['completion_date']) - pd.to_datetime(complaints['creation_date']))/86400000000000

In [111]:
select = complaints['Response Time'] > 0
complaints2 = complaints[select]

In [119]:
avg_response = complaints2[['Response Time','community']].groupby('community').mean().sort_values('Response Time')
print(avg_response[:5])
print(avg_response[-5:])

                 Response Time
community                     
LINCOLN PARK          4.271756
MCKINLEY PARK         4.320743
BRIGHTON PARK         4.521989
WEST TOWN             4.683101
LOWER WEST SIDE       4.829618
                Response Time
community                    
JEFFERSON PARK      30.664671
NORWOOD PARK        30.697206
OHARE               34.905000
EDISON PARK         35.469083
RIVERDALE           37.072917


In [122]:
avg_response['index'] = avg_response.index
area_names3 = area_names.merge(avg_response, left_on='community', right_on='index')
area_names3.columns
area_names3.plot(column='Response Time', cmap='Blues')
pyplot.show()

### Problem 2

In [30]:
# Adding Block ID to 311 Requests data 

#this file contains polygons at the FIPS_12 level (block group)
blocks = gpd.read_file('./data/cb_2015_17_bg_500k/cb_2015_17_bg_500k.shp')

data_with_blockid = sjoin(geo_complaints, blocks, how="left")

In [83]:
# Getting census data
def get_data_census(varname, label):

    census_api_url = 'http://api.census.gov/data/' + \
                    '2015/acs5?get=NAME,' + varname + \
                    '&for=block+group:*&in=state:17&in=county:031&in=tract:*'

    response = requests.get(census_api_url)

    json_data = response.content

    data = pd.read_json(json_data)

    names = list(data.iloc[0])

    for i in range(0,6):
        data.rename(index=str, columns={i: names[i]}, inplace=True)
    
    data.rename(index=str, columns={varname: label}, inplace=True)
    
    #HERE, I REPLICATE THE FIPS_12 CODE
    data['GEOID'] = data['state'].map(str) + data['county'].map(str) + \
                            data['tract'].map(str) + data['block group'].map(str)

    data.drop(data.index[[0]], inplace=True)
    
    print('downloaded', varname)
    
    return data

download_data = [('B01003_001E', 'Total Population'),
                ('B02001_002E', 'White Population'),
                ('B19013_001E', 'Median Household Income'),
                ('B05001_006E', 'Not a US Citizen'),
                ('B08303_001E', 'Tot time'),
                ('B08303_008E', 't30-34'),
                ('B08303_009E', 't35-39'),
                ('B08303_010E', 't40-44'),
                ('B08303_011E', 't45-59'),
                ('B08303_012E', 't60-89'),
                ('B08303_013E', 't90')]


for i, (varname, label) in enumerate(download_data):
    downloaded = get_data_census(varname, label)
    if i == 0:
        census_data = downloaded
    else:
        census_data = census_data.merge(downloaded, on='GEOID', how='outer')

data_with_census = data_with_blockid.merge(census_data, on='GEOID')    

downloaded B01003_001E
downloaded B02001_002E
downloaded B19013_001E
downloaded B05001_006E
downloaded B08303_001E
downloaded B08303_008E
downloaded B08303_009E
downloaded B08303_010E
downloaded B08303_011E
downloaded B08303_012E
downloaded B08303_013E


In [94]:
n_per_type_block = data_with_blockid[['GEOID', 'Complaint Type', 'service_request_number']].groupby(['GEOID', 'Complaint Type'], as_index=False).count()
n_per_type_block = n_per_type_block.pivot(index='GEOID', columns='Complaint Type', values='service_request_number')

def table_census_top5_blocks(ctype):
    a = n_per_type_block.sort_values(ctype, ascending=False)
    select = pd.DataFrame(list(a[:5].index))

    select_data = census_data.merge(select, left_on='GEOID', right_on=0)

    result = pd.DataFrame()
    result['Percent of White'] = pd.to_numeric(select_data['White Population']) / pd.to_numeric(select_data['Total Population'])
    result['Median Household Income'] = select_data['Median Household Income']
    print(select)
    result['Time'] = (pd.to_numeric(select_data['t40-44']) +
                      pd.to_numeric(select_data['t45-59']) + 
                      pd.to_numeric(select_data['t60-89']) + 
                      pd.to_numeric(select_data['t90'])) / pd.to_numeric(select_data['Tot time'])
    print(result)

#table_census_top5_blocks('Graffiti Removal')
table_census_top5_blocks('Vacant and Abandoned Buildings Reported')
table_census_top5_blocks('Sanitation Code Complaints')



              0
0  170314207004
1  170316713001
2  170314701001
3  170316711001
4  170314909022
   Percent of White Median Household Income      Time
0          0.044384                   21940  0.730994
1          0.000000                   31000  0.529412
2          0.007021                   29872  0.513595
3          0.002761                   25625  0.497817
4          0.000946                   35404  0.572308
              0
0  170315703004
1  170315703003
2  170311606023
3  170311606021
4  170315702001
   Percent of White Median Household Income      Time
0          0.700794                   39211  0.354749
1          0.738433                   54561  0.363330
2          0.715989                   41607  0.235079
3          0.540487                   37222  0.444934
4          0.489763                   52847  0.355000


### Problem 3

In [ ]:
import requests
from geopy.geocoders import Nominatim
import xml.etree.ElementTree as ET

address = '"7500 S. Wolcott Ave.'
geolocator = Nominatim()
location = geolocator.geocode(address)
print(location.latitude, location.longitude)

census_block_url = 'http://data.fcc.gov/api/block/2010/find?latitude={lat}&longitude={lon}&showall=true'.format(
        lat=location.latitude, lon=location.longitude)

response = requests.get(census_block_url)

xml_data = response.content
etree = ET.fromstring(xml_data)
#print(etree)

for neighbor in etree.iter():
    if 'Block' in neighbor.tag:
        block_id = neighbor.attrib['FIPS'][:12]

data_with_blockid2[data_with_blockid2.GEOID == block_id]["Complaint Type"].value_counts()


In [ ]:
result = data_with_blockid2[['community', 'Complaint Type', 'service_request_number']].groupby(['community', 'Complaint Type'], as_index=False).count()
select = result[result['Complaint Type'] == 'Graffiti Removal']
select

In [ ]:
select[select.community.str.contains('LAWNDALE')]

In [ ]:
select[select.community.str.contains('UPTOWN')]

In [ ]:
print(1778 / 116981)

In [ ]:
len(blocks2.blockce10.value_counts())

In [ ]:
blocks2.columns

In [ ]:
blocks2.countyfp10.value_counts()

In [ ]:
#to make graphs
%matplotlib inline
import matplotlib
blocks.plot();

base = blocks2.plot(color='white')

geo_sel.plot(ax=base, marker='o', color='red', markersize=5);